# Tutorial Overview
This tutorial (`uncertaintyforest_fig2.ipynb`) will further explain the UncertaintyForest class by allowing you to generate Figure 2 from [this paper](https://arxiv.org/pdf/1907.00325.pdf) on your own machine. 

If you haven't seen it already, take a look at other tutorials to setup and install the progressive learning package `Installation-and-Package-Setup-Tutorial.ipynb`

# Analyzing the UncertaintyForest Class by Reproducing Figure 2
## *Goal: Run the UncertaintyForest class to produce the results from Figure 2*
*Note: Figure 2 refers to Figure 2 from [this paper](https://arxiv.org/pdf/1907.00325.pdf)*

### First, we'll import the required packages

In [ ]:
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV

from proglearn.forest import UncertaintyForest
from functions.unc_forest_tutorials_functions import generate_data_fig2, CART_estimate, true_cond_entropy, format_func, get_cond_entropy_vs_n, get_cond_entropy_vs_mu, plot_cond_entropy_by_n, plot_cond_entropy_by_mu, plot_fig2

### Now, we'll specify some parameters 

In [ ]:
# The following are two sets of parameters.
# The first are those that were actually used to produce figure 2.
# These take a long time to actually run since there are up to 6000 data points.
# Below those, you'll find some testing parameters so that you can see the results more quickly.

# Here are the "Real Parameters"
# mus = [i * 0.5 for i in range(1, 11)] 
# effect_size = 1
# d1 = 1 
# d2 = 20 
# n1 = 3000 
# n2 = 6000 
# num_trials = 20 
# num_plotted_trials = 10 
# sample_sizes_d1 = range(300, 1201, 90) 
# sample_sizes_d2 = range(500, 3001, 250)

# Here are the "Test Parameters"
mus = [i * 0.5 for i in range(1, 3)] # range of means of the data (x-axis in right column)
effect_size = 1 # mu for left column
d1 = 1 # data dimensions = 1
d2 = 3 # data dimensions = 1, noise dimensions = 19
n1 = 100 # number of data points for top row, right column (d1)
n2 = 110 # number of data points for bottom row, right column (d2)
num_trials = 2 # number of trials to run
num_plotted_trials = 2 # the number of "fainter" lines to be displayed on the figure
sample_sizes_d1 = range(100, 120, 10) # range of data points for top row, left column (d1)
sample_sizes_d2 = range(100, 130, 10) # range of data points for bottom row, left column (d1)


In [ ]:
### In this function, we define how conditional entropy is estimated for the three kinds of learners used in Figure 2 (CART, IRF, and UF).

In [ ]:
def estimate_ce(X, y, label):
    if label == "CART":
        return CART_estimate(X, y)
    elif label == "IRF":
        frac_eval = 0.3
        irf = CalibratedClassifierCV(base_estimator=RandomForestClassifier(n_estimators = 300), 
                                     method='isotonic', 
                                     cv = 5)
        # X_train, y_train, X_eval, y_eval = split_train_eval(X, y, frac_eval)
        X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=frac_eval)
        irf.fit(X_train, y_train)
        p = irf.predict_proba(X_eval)
        return np.mean(entropy(p.T, base = np.exp(1)))
    elif label == "UF":
        frac_eval = 0.3
        uf = UncertaintyForest(n_estimators = n_estimators, tree_construction_proportion = 0.4, kappa = 3.0)
        # X_train, y_train, X_eval, y_eval = split_train_eval(X, y, frac_eval)
        X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=frac_eval)
        p = uf.predict_proba(X_eval)
        return np.mean(entropy(p.T, base = np.exp(1)))
    else:
        raise ValueError("Unrecognized Label!")

### Now, we'll specify which learners we'll compare (by label). Figure 2 uses three different learners specified below.

In [ ]:
# Algorithms used to produce figure 2
algos = [
    {
        'label': 'CART',
        'title': 'CART Forest',
        'color': "#1b9e77",
    },
    {
        'label': 'IRF',
        'title': 'Isotonic Reg. Forest',
        'color': "#fdae61",
    },
    {
        'label': 'UF',
        'title': 'Uncertainty Forest',
        'color': "#F41711",
    },
]

### Now, we'll run the code to obtain the results that will be displayed in Figure 2 (4 sets of calculations for 4 subplots)

In [ ]:
# This is the code that actually generates data and predictions for conditional entropy vs. n or mu.
get_cond_entropy_vs_n(effect_size, d1, num_trials, sample_sizes_d1, algos)

In [ ]:
get_cond_entropy_vs_mu(n1, d1, num_trials, mus, algos)

In [ ]:
get_cond_entropy_vs_n(effect_size, d2, num_trials, sample_sizes_d2, algos)

In [ ]:
get_cond_entropy_vs_mu(n2, d2, num_trials, mus, algos)

### Finally, create Figure 2.

In [ ]:
plot_fig2(num_plotted_trials, d1, d2, n1, n2, effect_size, algos)